In [50]:
import psycopg2
import pandas as pd

In [51]:
conn = psycopg2.connect(f"dbname=postgres user=postgres port=5433")
conn.autocommit = True
cur = conn.cursor()

In [52]:
# CREATE TABLE test_table 
# (
# name text,
# high_dv_val numeric,
# low_dv_val numeric,
# pnumber int
# )


# INSERT INTO test_table ( 
#   name,
#   high_dv_val,
#   low_dv_val,
#   pnumber
# ) 
# SELECT
#   (ARRAY['Curry','Green','Green','Thompson','Curry','Curry','Curry','Curry'])[floor(random() * 8 + 1)],
#   floor(random() * 100 + 1)::int,
#   floor(random() * 20 + 1)::int,
#   (ARRAY[1,1,1,1,1,1,2,2,3,4])[floor(random() * 10 + 1)]
#   FROM generate_series(1,10000)
#   ORDER BY random()
# ;


In [53]:
q = "SELECT * FROM test_table limit 10"
df = pd.read_sql(q, conn)
df.head()

,name,high_dv_val,low_dv_val,pnumber
0,Curry,1.0,1.0,1
1,Curry,1.0,1.0,1
2,Curry,1.0,1.0,1
3,Curry,1.0,1.0,1
4,Curry,1.0,1.0,1


In [54]:
std_q = "SELECT stddev(high_dv_val), name FROM test_table GROUP BY name"
df_std = pd.read_sql(std_q,conn)
df_std

,stddev,name
0,30.436561,Curry
1,3.670180,Thompson
2,7.297044,Green


In [55]:
sample_01_q = """
CREATE materialized view s1 as WITH text_weight AS
(SELECT name, COUNT(*) AS cnt, stddev(high_dv_val) AS sdv
FROM test_table
GROUP BY name
)
SELECT tt.* 
FROM test_table tt, text_weight tw
WHERE tt.name = tw.name
ORDER BY RANDOM() * tw.cnt * tw.sdv
LIMIT 1000; 
"""

sample_03_q = """
CREATE materialized view s3 as WITH text_weight AS
(SELECT name, COUNT(*) AS cnt, stddev(high_dv_val) AS sdv
FROM test_table
GROUP BY name
)
SELECT tt.* 
FROM test_table tt, text_weight tw
WHERE tt.name = tw.name
ORDER BY RANDOM() * tw.cnt * tw.sdv
LIMIT 3000; 
"""

sample_05_q = """
CREATE materialized view s5 as WITH text_weight AS
(SELECT name, COUNT(*) AS cnt, stddev(high_dv_val) AS sdv
FROM test_table
GROUP BY name
)
SELECT tt.* 
FROM test_table tt, text_weight tw
WHERE tt.name = tw.name
ORDER BY RANDOM() * tw.cnt * tw.sdv
LIMIT 5000; 
"""

sample_07_q = """
CREATE materialized view s7 as WITH text_weight AS
(SELECT name, COUNT(*) AS cnt, stddev(high_dv_val) AS sdv
FROM test_table
GROUP BY name
)
SELECT tt.* 
FROM test_table tt, text_weight tw
WHERE tt.name = tw.name
ORDER BY RANDOM() * tw.cnt * tw.sdv
LIMIT 7000; 
"""

# cur.execute(sample_01_q)
# cur.execute(sample_03_q)
# cur.execute(sample_05_q)
# cur.execute(sample_07_q)

In [56]:
random_test_patterns_q = """
SELECT * FROM test_table
ORDER BY RANDOM()
LIMIT 100;
"""

test_sample_patterns = pd.read_sql(random_test_patterns_q, conn).to_dict('records')

In [57]:
test_sample_patterns

[{'name': 'Curry', 'high_dv_val': 3.0, 'low_dv_val': 1.0, 'pnumber': 1},
 {'name': 'Green', 'high_dv_val': 23.0, 'low_dv_val': 5.0, 'pnumber': 1},
 {'name': 'Curry', 'high_dv_val': 65.0, 'low_dv_val': 13.0, 'pnumber': 2},
 {'name': 'Curry', 'high_dv_val': 11.0, 'low_dv_val': 3.0, 'pnumber': 1},
 {'name': 'Curry', 'high_dv_val': 77.0, 'low_dv_val': 16.0, 'pnumber': 2},
 {'name': 'Green', 'high_dv_val': 27.0, 'low_dv_val': 6.0, 'pnumber': 1},
 {'name': 'Curry', 'high_dv_val': 67.0, 'low_dv_val': 14.0, 'pnumber': 2},
 {'name': 'Curry', 'high_dv_val': 1.0, 'low_dv_val': 1.0, 'pnumber': 1},
 {'name': 'Curry', 'high_dv_val': 90.0, 'low_dv_val': 18.0, 'pnumber': 3},
 {'name': 'Curry', 'high_dv_val': 78.0, 'low_dv_val': 16.0, 'pnumber': 2},
 {'name': 'Curry', 'high_dv_val': 11.0, 'low_dv_val': 3.0, 'pnumber': 1},
 {'name': 'Curry', 'high_dv_val': 54.0, 'low_dv_val': 11.0, 'pnumber': 1},
 {'name': 'Green', 'high_dv_val': 31.0, 'low_dv_val': 7.0, 'pnumber': 1},
 {'name': 'Green', 'high_dv_val': 

In [58]:
samples = ['s1', 's3', 's5', 's7', 'test_table']

for p in test_sample_patterns:
    for s in samples:
        sample_eval_q = f"""
        WITH tp AS 
        (
        SELECT COUNT(DISTINCT pnumber) AS tp
        FROM {s}
        WHERE name = '{p["name"]}'
        AND high_dv_val = {p['high_dv_val']}
        AND low_dv_val = {p['low_dv_val']}
        )
        SELECT t.tp::numeric/COUNT(DISTINCT pnumber)
        FROM {s}, tp t
        GROUP BY t.tp
        """
        cur.execute(sample_eval_q)
        p[s] = float(cur.fetchone()[0])
result_df = pd.DataFrame(test_sample_patterns)

result_df

,name,high_dv_val,low_dv_val,pnumber,s1,s3,s5,s7,test_table
0,Curry,3.0,1.0,1,0.25,0.25,0.25,0.25,0.25
1,Green,23.0,5.0,1,0.25,0.25,0.25,0.25,0.25
2,Curry,65.0,13.0,2,0.25,0.25,0.25,0.25,0.25
3,Curry,11.0,3.0,1,0.00,0.25,0.25,0.25,0.25
4,Curry,77.0,16.0,2,0.25,0.25,0.25,0.25,0.25
...,...,...,...,...,...,...,...,...,...
95,Curry,2.0,1.0,1,0.25,0.25,0.25,0.25,0.25
96,Green,21.0,5.0,1,0.25,0.25,0.25,0.25,0.25
97,Curry,91.0,19.0,4,0.25,0.25,0.25,0.25,0.25
98,Curry,72.0,15.0,2,0.00,0.25,0.25,0.25,0.25


In [59]:
result_df[(result_df['s1']!=result_df['test_table'])]

,name,high_dv_val,low_dv_val,pnumber,s1,s3,s5,s7,test_table
3,Curry,11.0,3.0,1,0.0,0.25,0.25,0.25,0.25
8,Curry,90.0,18.0,3,0.0,0.25,0.25,0.25,0.25
10,Curry,11.0,3.0,1,0.0,0.25,0.25,0.25,0.25
22,Curry,93.0,19.0,4,0.0,0.25,0.25,0.25,0.25
26,Curry,97.0,20.0,4,0.0,0.25,0.25,0.25,0.25
27,Curry,68.0,14.0,2,0.0,0.25,0.25,0.25,0.25
31,Curry,87.0,18.0,3,0.0,0.25,0.25,0.25,0.25
37,Curry,86.0,18.0,3,0.0,0.25,0.25,0.25,0.25
49,Curry,98.0,20.0,4,0.0,0.25,0.25,0.25,0.25
52,Curry,86.0,18.0,3,0.0,0.25,0.25,0.25,0.25


In [60]:
result_df[(result_df['s3']!=result_df['test_table'])]

,name,high_dv_val,low_dv_val,pnumber,s1,s3,s5,s7,test_table


In [61]:
result_df[(result_df['s5']!=result_df['test_table'])]

,name,high_dv_val,low_dv_val,pnumber,s1,s3,s5,s7,test_table


In [62]:
result_df[(result_df['s7']!=result_df['test_table'])]

,name,high_dv_val,low_dv_val,pnumber,s1,s3,s5,s7,test_table
